### Colab

In [2]:
!git clone https://github.com/aivion2k/ZUM_2025_MichalKrol.git zum_mnist_mk2
%cd zum_mnist_mk2

!pip -q install --upgrade pip
!pip -q install pytorch-lightning transformers torchmetrics scikit-learn

import pytorch_lightning as pl
import torch
print("PL:", pl.__version__)
print("CUDA:", torch.cuda.is_available())

Cloning into 'zum_mnist_mk2'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 40 (delta 8), reused 34 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (40/40), 353.82 KiB | 8.84 MiB/s, done.
Resolving deltas: 100% (8/8), done.
/content/zum_mnist_mk/zum_mnist_mk2
PL: 2.6.0
CUDA: True


### Trening

In [3]:
import sys
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()
sys.path.append(str(PROJECT_ROOT))
sys.path.append(str(PROJECT_ROOT / "src"))

In [5]:
import pytorch_lightning as pl
import numpy as np
import matplotlib.pyplot as plt

from src.data.datamodule import MNISTDataModule
from src.modules.transformer import ViTLightning, early_stop
import torch

MODEL_NAME = "facebook/deit-tiny-patch16-224"

dm = MNISTDataModule(for_transformer=True, batch_size=64, num_workers=2)
model = ViTLightning(model_name=MODEL_NAME, lr=1e-5, wd=0.05)

trainer = pl.Trainer(
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1,
    max_epochs=20,
    precision="16-mixed" if torch.cuda.is_available() else "32",
    callbacks=[early_stop],
    logger=False,
    enable_checkpointing=False,
)

trainer.fit(model, datamodule=dm)
trainer.test(model, datamodule=dm)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at facebook/deit-tiny-patch16-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 192]) in the checkpoint and torch.Size([10, 192]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name      ┃ Type                      ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ model     │ ViTForImageClassification │  5.5 M │ eval  │     0 │
│ 1 │ train_acc │ MulticlassAccuracy        │      0 │ train │     0 │
│ 2 │ val_acc   │ MulticlassAccuracy        │      0 │ train │     0 │
│ 3 │ test_acc  │ MulticlassAccuracy        │      0 │ train │     0 │
└───┴───────────┴───────────────────────────┴────────┴───────┴───────┘

Trainable params: 5.5 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 5.5 M                                                                                                
Total estimated model params size (MB): 22                                                                         
Modules in train mode: 3                                                                                           
Modules in eval mode: 214                                                                                          
Total FLOPs: 0

Output()

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9890145659446716     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc': 0.9890145659446716}]